# Scraping real state attributes in Benito Juarez, CDMX

In [1]:
from bs4 import BeautifulSoup
import requests
from scrapy import Selector
import pandas as pd
import numpy as np
#my functions
from rdb_functions import clean_list
from rdb_functions import clean_list_url


We manually searched real state sales in metroscubicos.com in Alcaldia Benito Juarez.

In [2]:
url='https://inmuebles.metroscubicos.com/casas/venta/distrito-federal/benito-juarez/#origin=search&as_word=true'
#Get html content
html = requests.get( url ).content
sel = Selector( text = html )

## Getting all urls

In [3]:
path='//*[@class="andes-pagination__link"]'
urls=sel.xpath(path).extract()

#Getting the fist urls
urls=clean_list_url(urls)

there aren't further urls beyond the last from the previous query

In [4]:
#starting in last position from the previous query
url=urls[len(urls)-1]

html = requests.get( url ).content
sel = Selector( text = html )

#Getting the lasts urls
path='//*[@class="andes-pagination__link"]'
urls2=sel.xpath(path).extract()
urls2=clean_list_url(urls2)
urls.extend(urls2)

## Getting Attributes

In [5]:
#empty list of attributes
atr=[]
price=[]
status=[]
adress=[]

for url in urls:
    try:
        html = requests.get( url ).content
        sel = Selector( text = html )
    except:
        print(f'This url didnt work: \n {url}')
              
    try:
        path='//*[@class="item__attrs"]'
        aux=sel.xpath(path).extract()
        atr.extend(clean_list(aux))
    except:
        print(f'no attributes info in:\n{url}')
        
    try:
        path='//*[@class="price__fraction"]'
        aux=sel.xpath(path).extract()
        price.extend(clean_list(aux))
    except:
        print(f'no prices info in:\n{url}')

    try:
        path='//*[@class="item__info-title"]'
        aux=sel.xpath(path).extract()
        status.extend(clean_list(aux))
    except:
        print(f'no prices info in:\n{url}')
        
    try:
        path='//*[@class="item__title"]'
        aux=sel.xpath(path).extract()
        adress.extend(clean_list(aux))
    except:
        print(f'no prices info in:\n{url}')  
        

This url didnt work: 
 #
This url didnt work: 
 #


## Generating the Pandas

In [6]:
data=pd.DataFrame({'attributes':atr,'price':price,'status':status,'adress':adress})

In [7]:
data.tail(10)

,attributes,price,status,adress
998,140 m² construidos | 2 recs.,"5,900,000",Casa en venta,Alhambra 1011 - Portales Sur - Benito Juárez ...
999,124 m² construidos | 3 recs.,"2,700,000",Casa en venta,Castilla - Benito Juárez - Distrito Federal
1000,247 m² construidos | 3 recs.,"15,219,666",Casa en venta,Amores - Del Valle Norte - Benito Juárez - Di...
1001,180 m² construidos | 3 recs.,"1,155,966",Casa en venta,CASTILLA 142 - Alamos - Benito Juárez - Distr...
1002,150 m² construidos | 3 recs.,"1,400,000",Casa en venta,MANUEL LOPEZ COTILLA 934 - Del Valle Centro -...
1003,180 m² construidos | 4 recs.,"1,352,036",Casa en venta,GABRIEL MANCER 46 - Del Valle Centro - Benito...
1004,150 m² construidos | 3 recs.,"5,990,000",Casa en venta,Jaca 914 - Santa Cruz Atoyac - Benito Juárez ...
1005,560 m² construidos | 4 recs.,"1,010,000",Casa en venta,IDAHO 00000 - Nápoles - Benito Juárez - Distr...
1006,200 m² construidos | 4 recs.,"1,049,998",Casa en venta,MARIA HERNANDEZ ZARCO 68 - Alamos - Benito Ju...
1007,219 m² construidos | 3 recs.,"13,686,875",Casa en venta,Dallas - Nápoles - Benito Juárez - Distrito F...


## Printing the results

In [8]:
data.to_csv('benito_juarez.csv',index=False)